<a href="https://colab.research.google.com/github/diogodsa/Stock-Investing-Using-Machine-Learning/blob/master/Stock_Investing_Portfolio_Using_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##A primeira etapa é importar todas as bibliotecas necessárias.

In [3]:
pip install yahoo_fin

In [87]:
import requests
import pandas as pd 
from yahoo_fin import stock_info as si 
from pandas_datareader import DataReader
import numpy as np

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
tickers = si.tickers_sp500()
recommendations = []

for ticker in tickers:
    lhs_url = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
    rhs_url = '?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&' \
              'modules=upgradeDowngradeHistory,recommendationTrend,' \
              'financialData,earningsHistory,earningsTrend,industryTrend&' \
              'corsDomain=finance.yahoo.com'
              
    url =  lhs_url + ticker + rhs_url
    r = requests.get(url)
    if not r.ok:
        recommendation = 6
    try:
        result = r.json()['quoteSummary']['result'][0]
        recommendation =result['financialData']['recommendationMean']['fmt']
    except:
        recommendation = 6
    
    recommendations.append(recommendation)
    print(ticker)

In [62]:
df = pd.DataFrame(list(zip(tickers, recommendations)), 
               columns =['Company', 'Recommendations']) 
df 

,Company,Recommendations
0,A,2.20
1,AAL,3.30
2,AAP,2.30
3,AAPL,2.00
4,ABBV,2.00
...,...,...
500,YUM,2.60
501,ZBH,1.90
502,ZBRA,2.30
503,ZION,2.70


In [ ]:
arrRecommendations = np.array(df.Recommendations, dtype=float)

hold_df = df[arrRecommendations == 3]
buy_df = df[arrRecommendations <= 1.5]
sell_df = df[arrRecommendations >= 4.5]

df_list = [hold_df, buy_df, sell_df]
new_df = pd.concat(df_list)
new_df.reset_index(level=0, inplace=True)

new_df

In [ ]:
tickers =[] 
  
for index, rows in new_df.iterrows(): 
    tickers.append(rows.Company) 

tickers

In [88]:
finwiz_url = 'https://finviz.com/quote.ashx?t='

news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 

    try:
        response = urlopen(req)  
    except Exception:
        pass

    html = BeautifulSoup(response)
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table
    print(url)
    
parsed_news = []

for file_name, news_table in news_tables.items():
    try:
        for x in news_table.findAll('tr'):
    
          text = x.a.get_text() 
          date_scrape = x.td.text.split()

          if len(date_scrape) == 1:
              time = date_scrape[0]
              
          else:
              date = date_scrape[0]
              time = date_scrape[1]
          ticker = file_name.split('_')[0]
          
          parsed_news.append([ticker, date, time, text])
    except Exception:
        pass

    
        
vader = SentimentIntensityAnalyzer()

columns = ['ticker', 'date', 'time', 'headline']

parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)

parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date

parsed_and_scored_news.head()

https://finviz.com/quote.ashx?t=CINF
https://finviz.com/quote.ashx?t=CLX
https://finviz.com/quote.ashx?t=CMA
https://finviz.com/quote.ashx?t=COTY
https://finviz.com/quote.ashx?t=CTSH
https://finviz.com/quote.ashx?t=DISCK
https://finviz.com/quote.ashx?t=ITW
https://finviz.com/quote.ashx?t=JKHY
https://finviz.com/quote.ashx?t=KIM
https://finviz.com/quote.ashx?t=KSS
https://finviz.com/quote.ashx?t=L
https://finviz.com/quote.ashx?t=NTAP
https://finviz.com/quote.ashx?t=RHI
https://finviz.com/quote.ashx?t=UA
https://finviz.com/quote.ashx?t=UAA
https://finviz.com/quote.ashx?t=VTR
https://finviz.com/quote.ashx?t=WRB
https://finviz.com/quote.ashx?t=BF.B
https://finviz.com/quote.ashx?t=BRK.B
https://finviz.com/quote.ashx?t=FOX
https://finviz.com/quote.ashx?t=NWS


,ticker,date,time,headline
0,CINF,2020-06-19,11:24AM,Cincinnati Financial (CINF) Plunges 39.2% YTD:...
1,CINF,2020-06-08,12:48PM,3 Low Price-Book Stocks
2,CINF,2020-06-06,08:31AM,Here are the 10 most under and overbought S&P ...
3,CINF,2020-06-05,10:09AM,Hedge Funds Are Done Buying Cincinnati Financi...
4,CINF,2020-06-02,12:33PM,"Pandemic, Protests, and Hurricanes: Can It Get..."


In [89]:
parsed_and_scored_news = parsed_and_scored_news.groupby(['ticker'], as_index = False).agg({'headline': ''.join}, Inplace=True)

## Performing Sentiment Analysis

In [91]:
vader = SentimentIntensityAnalyzer()


scores = parsed_and_scored_news['headline'].apply(vader.polarity_scores).tolist()

scores_df = pd.DataFrame(scores)

parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')

parsed_and_scored_news

,ticker,headline,neg,neu,pos,compound,neg_right,neu_right,pos_right,compound_right
0,CINF,Cincinnati Financial (CINF) Plunges 39.2% YTD:...,0.025,0.897,0.078,0.9938,0.025,0.897,0.078,0.9938
1,CLX,Clorox (CLX) Outpaces Stock Market Gains: What...,0.050,0.830,0.120,0.9968,0.050,0.830,0.120,0.9968
2,CMA,Want Income? Nows the Time to Consider Preferr...,0.037,0.848,0.116,0.9966,0.037,0.848,0.116,0.9966
3,COTY,Coty Inc. -- Coty's operating risks and levera...,0.039,0.880,0.081,0.9901,0.039,0.880,0.081,0.9901
4,CTSH,5 Tech Companies Boosting Book ValueCDW: Does ...,0.046,0.840,0.114,0.9940,0.046,0.840,0.114,0.9940
5,DISCK,5 Undervalued Predictable Companies for Summer...,0.034,0.887,0.079,0.9898,0.034,0.887,0.079,0.9898
6,FOX,Bidens lead grows to 12 points in Fox News pol...,0.031,0.879,0.089,0.9961,0.031,0.879,0.089,0.9961
7,ITW,3 Top Industrial Stocks to Buy Right NowWhy Is...,0.023,0.850,0.127,0.9985,0.023,0.850,0.127,0.9985
8,JKHY,SAFE Federal Credit Union Launches Fully Virtu...,0.017,0.863,0.121,0.9984,0.017,0.863,0.121,0.9984
9,KIM,Hedge Funds Have Never Been This Bullish On Ki...,0.015,0.881,0.104,0.9984,0.015,0.881,0.104,0.9984
